# Bibliotecas

In [1]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

2024-03-07 20:50:35.105210: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 20:50:35.393744: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 20:50:35.393785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 20:50:35.445554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 20:50:35.534767: I tensorflow/core/platform/cpu_feature_guar

# Abrindo dados

In [2]:
x = np.float64(np.load(open('data/cnn/x.npy', 'rb')))
y = np.int8(np.load(open('data/cnn/y.npy', 'rb')))

In [3]:
from keras.utils import to_categorical

y_one_hot = to_categorical(y, 3)
y_one_hot

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

# Criar Modelo

In [5]:
modelo = Sequential()

modelo.add(Conv2D(256, (25, 5), activation='relu', input_shape=(501, 6, 1)))
modelo.add(MaxPooling2D((2, 2)))

modelo.add(Flatten())
modelo.add(Dense(1024, activation='relu'))
modelo.add(Dense(512, activation='relu'))
modelo.add(Dense(3, activation='softmax'))

modelo.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 477, 2, 256)       32256     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 238, 1, 256)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 60928)             0         
                                                                 
 dense (Dense)               (None, 1024)              62391296  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 3)                 1539      
                                                      

# Treinar Modelo

In [6]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

tensorboard_callback = TensorBoard(log_dir="./logs/cnn/")
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=20, min_delta=0.001, verbose=True)
modelcheckpoint_cb = ModelCheckpoint('models/CNNmodel.h5', 'val_accuracy', save_best_only=True)

modelo.fit(
    x,
    y_one_hot,
    shuffle=True, 
    epochs=200,
    batch_size=256,
    validation_split=0.2,
    callbacks=[tensorboard_callback, earlystop_cb, modelcheckpoint_cb]
)

2024-03-07 20:59:21.763419: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/200


2024-03-07 20:59:22.399502: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-07 20:59:22.646242: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-07 20:59:23.393411: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6505154050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-07 20:59:23.393442: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-07 20:59:23.405619: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709859563.497825    5611 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


234/234 [==============================] - ETA: 0s - loss: 1.0265 - accuracy: 0.4614

/home/aurelio/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


234/234 [==============================] - 18s 66ms/step - loss: 1.0265 - accuracy: 0.4614 - val_loss: 0.9790 - val_accuracy: 0.4782
Epoch 2/200
234/234 [==============================] - 12s 50ms/step - loss: 0.9372 - accuracy: 0.5409 - val_loss: 0.8695 - val_accuracy: 0.5922
Epoch 3/200
234/234 [==============================] - 12s 50ms/step - loss: 0.7455 - accuracy: 0.6751 - val_loss: 0.6425 - val_accuracy: 0.7328
Epoch 4/200
234/234 [==============================] - 12s 50ms/step - loss: 0.5134 - accuracy: 0.7980 - val_loss: 0.4852 - val_accuracy: 0.8121
Epoch 5/200
234/234 [==============================] - 12s 51ms/step - loss: 0.3746 - accuracy: 0.8579 - val_loss: 0.4017 - val_accuracy: 0.8526
Epoch 6/200
234/234 [==============================] - 12s 50ms/step - loss: 0.2981 - accuracy: 0.8909 - val_loss: 0.3532 - val_accuracy: 0.8729
Epoch 7/200
234/234 [==============================] - 12s 50ms/step - loss: 0.2592 - accuracy: 0.9034 - val_loss: 0.3398 - val_accuracy: 0.87

In [ ]:
# modelo.save('models/tf-cnn-model')